In [ ]:
import pandas as pd
import lightningchart as lc
import numpy as np

lc.set_license('my-license-key')

In [3]:
file_path = "../Dataset/gold_recovery_full_with_profit.csv"
df = pd.read_csv(file_path)
df.head()

,date,final.output.concentrate_ag,final.output.concentrate_pb,final.output.concentrate_sol,final.output.concentrate_au,final.output.recovery,final.output.tail_ag,final.output.tail_pb,final.output.tail_sol,final.output.tail_au,...,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level,Close,profit
0,1/15/2016,6.055403,9.889648,5.507324,42.192020,70.541216,10.411962,0.895447,16.904297,2.143149,...,12.099931,-504.715942,9.925633,-498.310211,8.079666,-500.470978,14.151341,-605.841980,107.89,4552.097083
1,1/15/2016,6.029369,9.968944,5.257781,42.701629,69.266198,10.462676,0.927452,16.634514,2.224930,...,11.950531,-501.331529,10.039245,-500.169983,7.984757,-500.582168,13.998353,-599.787184,107.89,4607.078779
2,1/15/2016,6.055926,10.213995,5.383759,42.657501,68.116445,10.507046,0.953716,16.208849,2.257889,...,11.912783,-501.133383,10.070913,-500.129135,8.013877,-500.517572,14.028663,-601.427363,107.89,4602.317741
3,1/15/2016,6.047977,9.977019,4.858634,42.689819,68.347543,10.422762,0.883763,16.532835,2.146849,...,11.999550,-501.193686,9.970366,-499.201640,7.977324,-500.255908,14.005551,-599.996129,107.89,4605.804622
4,1/15/2016,6.148599,10.142511,4.939416,42.774141,66.927016,10.360302,0.792826,16.525686,2.055292,...,11.953070,-501.053894,9.925709,-501.686727,7.894242,-500.356035,13.996647,-601.496691,107.89,4614.902068


In [4]:
import time
import math
import numpy as np
import pandas as pd
import lightningchart as lc
import keyboard  # pip install keyboard
import sys

# -----------------------
# --- Data Preparation ---
# -----------------------
# Load your dataset (for example, gold_recovery_full.csv)
# Here we assume that df is already loaded.
# Ensure numeric types for the relevant columns.
df["TrialNumber"] = df.index  # using index as trial number
df["profit"] = df["profit"].astype(float)  # profit column replaces Profit(USD)
for col in [
    "final.output.concentrate_ag",
    "final.output.concentrate_pb",
    "final.output.concentrate_sol",
    "final.output.concentrate_au"
]:
    df[col] = df[col].astype(float)
    
x_values = df["TrialNumber"].tolist()

# -----------------------------
# --- Star Event Preparation ---
# -----------------------------
star_added = set()       # indices already processed as stars
last_star_index = None   # previous star index for consecutive comparison
best_profit_delta = -sys.maxsize  # store highest profit delta seen so far
best_combo_values = {}   # store best combination (deltas) for features

# ------------------------
# --- Dashboard Setup ---
# ------------------------
# Create an 8x8 grid dashboard.
dashboard = lc.Dashboard(theme=lc.Themes.White, rows=8, columns=8)

# --- Live Streaming Chart (Left Panel) ---
# Display the five series: 4 concentrate features and profit.
live_chart = dashboard.ChartXY(
    title="Live Streaming: Concentrate & Profit",
    row_index=0, column_index=0, row_span=8, column_span=4
)
# (Optionally hide ticks if desired.)
live_chart.get_default_y_axis().set_tick_strategy("Empty")
legend = live_chart.add_legend()

# New list of series for live streaming.
y_columns = [
    "final.output.concentrate_ag",
    "final.output.concentrate_pb",
    "final.output.concentrate_sol",
    "final.output.concentrate_au",
    "profit"
]

axis_labels = {
    "final.output.concentrate_ag": "ag",
    "final.output.concentrate_pb": "pb",
    "final.output.concentrate_sol": "sol",
    "final.output.concentrate_au": "au",
    "profit": "profit"
}

series_dict = {}
min_star_series_dict = {}
max_star_series_dict = {}
live_chart.get_default_y_axis().dispose()
for idx, y_col in enumerate(y_columns):
    axis_y = live_chart.add_y_axis(stack_index=idx)
    short_title = axis_labels.get(y_col, y_col)
    axis_y.set_title(short_title).set_title_rotation(0)
    
    min_value = df[y_col].min()
    max_value = df[y_col].max()
    diff = max_value - min_value
    
    line_series = live_chart.add_line_series(y_axis=axis_y, data_pattern="ProgressiveX").set_effect(False)
    line_series.set_name(short_title)
    legend.add(line_series).set_dragging_mode("draggable")
    series_dict[y_col] = line_series
    
    # Create star series for minima (red) and maxima (green)
    min_star_series = live_chart.add_point_series(y_axis=axis_y)
    min_star_series.set_point_shape('star')
    min_star_series.set_point_color(lc.Color("red"))
    min_star_series.set_point_size(10)
    min_star_series.set_name(f"{y_col} Min Stars")
    min_star_series_dict[y_col] = min_star_series
    
    max_star_series = live_chart.add_point_series(y_axis=axis_y)
    max_star_series.set_point_shape('star')
    max_star_series.set_point_color(lc.Color("green"))
    max_star_series.set_point_size(10)
    max_star_series.set_name(f"{y_col} Max Stars")
    max_star_series_dict[y_col] = max_star_series

live_chart.get_default_x_axis().set_title("Trial Number")

# --- Waterfall Chart (Top Right Panel) ---
waterfall_chart = dashboard.BarChart(
    vertical=True,
    row_index=0, column_index=4, row_span=6, column_span=4
)
waterfall_chart.set_sorting('disabled').set_title("Impact of Concentrate Changes on Profit")
waterfall_chart.set_data([]).set_label_rotation(45)

# --- Best Combination Textboxes (Bottom Right Panel) ---
# We now have 5 features: 4 concentrate features and profit.
# We will arrange the text boxes in two rows: top row for the first three, bottom row for the remaining two.
best_combo_chart = dashboard.ChartXY(
    row_index=6, column_index=4, row_span=2, column_span=4
)
best_combo_chart.set_title("Best Combination of Feature Changes")
best_combo_chart.get_default_x_axis().set_tick_strategy("Empty")
best_combo_chart.get_default_y_axis().set_tick_strategy("Empty")

# Define new features list.
features = [
    "final.output.concentrate_ag",
    "final.output.concentrate_pb",
    "final.output.concentrate_sol",
    "final.output.concentrate_au",
    "profit"
]

# Define positions (x,y in percentages) for value text boxes.
# Top row: first three features; bottom row: last two.
textbox_positions = {
    "final.output.concentrate_ag": (20, 68),
    "final.output.concentrate_pb": (50, 68),
    "final.output.concentrate_sol": (80, 68),
    "final.output.concentrate_au": (35, 25),
    "profit": (65, 25)
}

# Define icon text box positions by shifting x by -10%.
icon_textbox_positions = { 
    feature: (max(pos[0] - 10, 0), pos[1]) 
    for feature, pos in textbox_positions.items()
}

# Create dictionaries to hold the text box objects.
best_combo_textboxes = {}
best_combo_icon_textboxes = {}

# Add value text boxes.
for feature in features:
    short_label = axis_labels.get(feature, feature)
    pos = textbox_positions[feature]
    tb = best_combo_chart.add_textbox(f"{short_label}: ---", x=pos[0], y=pos[1], position_scale="percentage")
    tb.set_text_font(16, weight="bold")
    tb.set_stroke(thickness=0, color=lc.Color(0, 0, 0, 0))
    best_combo_textboxes[feature] = tb

# Add icon text boxes.
for feature in features:
    pos = icon_textbox_positions[feature]
    tb_icon = best_combo_chart.add_textbox("", x=pos[0], y=pos[1], position_scale="percentage")
    tb_icon.set_text_font(16, weight="bold")
    tb_icon.set_stroke(thickness=0, color=lc.Color(0, 0, 0, 0))
    best_combo_icon_textboxes[feature] = tb_icon

# ---------------------------
# --- Open the Dashboard ---
# ---------------------------
dashboard.open(live=True, method="browser")

# ------------------------------
# --- Pause/Resume Setup ---
# ------------------------------
paused = False
def toggle_pause():
    global paused
    paused = not paused
    if paused:
        print("Streaming paused. Press 'p' again to resume.")
    else:
        print("Streaming resumed.")
keyboard.add_hotkey("p", toggle_pause)

# -------------------------------------------
# --- Live Streaming Simulation Loop ---
# -------------------------------------------
n = len(df)
for i in range(n):
    # Check for pause.
    while paused:
        time.sleep(0.1)
    
    # Append new data points for each feature.
    for y_col in y_columns:
        new_x = float(df["TrialNumber"].iloc[i])  # Cast to float
        new_y = float(df[y_col].iloc[i])            # Cast to float
        series_dict[y_col].add([new_x], [new_y])
    
    # After at least 3 points, check for local extrema in profit.
    if i >= 2:
        prev_profit = float(df["profit"].iloc[i - 1])
        prev_prev_profit = float(df["profit"].iloc[i - 2])
        current_profit = float(df["profit"].iloc[i])
        
        if (i - 1) not in star_added:
            star_index = i - 1
            star_detected = False
            
            # Detect local minimum.
            if prev_profit < prev_prev_profit and prev_profit < current_profit:
                for y_col in y_columns:
                    star_x = float(df["TrialNumber"].iloc[star_index])
                    star_y = float(df[y_col].iloc[star_index])
                    min_star_series_dict[y_col].add([star_x], [star_y])
                star_detected = True
            # Detect local maximum.
            elif prev_profit > prev_prev_profit and prev_profit > current_profit:
                for y_col in y_columns:
                    star_x = float(df["TrialNumber"].iloc[star_index])
                    star_y = float(df[y_col].iloc[star_index])
                    max_star_series_dict[y_col].add([star_x], [star_y])
                star_detected = True
            
            if star_detected:
                star_added.add(star_index)
                
                # --- Compute Waterfall Data for Consecutive Stars ---
                # Use the four concentrate features for delta calculations.
                if last_star_index is not None:
                    waterfall_data = []
                    parameters = [
                        "final.output.concentrate_ag",
                        "final.output.concentrate_pb",
                        "final.output.concentrate_sol",
                        "final.output.concentrate_au"
                    ]

                    for param in parameters:
                        delta = df[param].iloc[star_index] - df[param].iloc[last_star_index]
                        short_label = axis_labels.get(param, param)
                        waterfall_data.append({"category": short_label, "value": float(delta)})

                    profit_delta = float(df["profit"].iloc[star_index] - df["profit"].iloc[last_star_index])
                    waterfall_data.append({"category": axis_labels.get("profit", "profit"), "value": profit_delta})

                    
                    waterfall_chart.set_data(waterfall_data)
                    print("Waterfall chart updated between trial:",
                          df["TrialNumber"].iloc[last_star_index],
                          "and",
                          df["TrialNumber"].iloc[star_index])
                    
                    # --- Update Best Combination Textboxes with Icons ---
                    # Update if this profit delta is the highest so far.
                    if profit_delta > best_profit_delta:
                        best_profit_delta = profit_delta
                        best_combo_values = {}
                        for param in parameters:
                            best_combo_values[param] = float(df[param].iloc[star_index] - df[param].iloc[last_star_index])
                        best_combo_values["profit"] = profit_delta
                        
                        # Update value text boxes.
                        for feature, tb in best_combo_textboxes.items():
                            short_label = axis_labels.get(feature, feature)
                            tb.set_text(f"{short_label}: {best_combo_values[feature]:.2f}")
                            if feature == "profit":
                                tb.set_stroke(thickness=2, color=(lc.Color("red")))
                        # Update icon text boxes based on sign.
                        for feature, tb_icon in best_combo_icon_textboxes.items():
                            if best_combo_values[feature] >= 0:
                                tb_icon.add_image("D:/Computer Aplication/WorkPlacement/Projects/Project22/Images/up2.png", fit_mode="Fit", size={"width": 20, "height": 20})
                            else:
                                tb_icon.add_image("D:/Computer Aplication/WorkPlacement/Projects/Project22/Images/down2.png", fit_mode="Fit", size={"width": 20, "height": 20})
                        print("Best combination updated between trial:",
                              df["TrialNumber"].iloc[last_star_index],
                              "and",
                              df["TrialNumber"].iloc[star_index],
                              "with profit delta:", profit_delta)
                
                # Update last_star_index for the next comparison.
                last_star_index = star_index
                
                # Sleep 2 seconds after processing a star event so the waterfall chart and text boxes can update.
                time.sleep(6)
    
    # Simulate streaming delay.
    time.sleep(1)


Waterfall chart updated between trial: 1 and 2
Best combination updated between trial: 1 and 2 with profit delta: -4.761038000000553
Waterfall chart updated between trial: 2 and 4
Best combination updated between trial: 2 and 4 with profit delta: 12.584327000000485
Waterfall chart updated between trial: 4 and 5
Waterfall chart updated between trial: 5 and 8
Best combination updated between trial: 5 and 8 with profit delta: 191.0084569999999
Waterfall chart updated between trial: 8 and 9
Waterfall chart updated between trial: 9 and 10


KeyboardInterrupt: 

Streaming paused. Press 'p' again to resume.
Streaming resumed.
Streaming paused. Press 'p' again to resume.
Streaming resumed.
Streaming paused. Press 'p' again to resume.
Streaming resumed.
Streaming paused. Press 'p' again to resume.
Streaming resumed.
Streaming paused. Press 'p' again to resume.
Streaming resumed.
Streaming paused. Press 'p' again to resume.
Streaming resumed.
